In [3]:
import sys
import os
import re
import pandas as pd

sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

from src.load_data import load_excel
from src.transform import rellenar_valores, estandarizar_diccionario
from src.translate_codes import traducir_columnas, traducir_encabezados

#Configuracion de Paths
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
print("BASE_DIR:", BASE_DIR)

#Ruta diccionarios
ruta_diccionario = os.path.join(BASE_DIR, "data", "raw", "diccionario.xlsx")

#Cargar tabla principal
df_datos = load_excel(os.path.join(BASE_DIR, "data", "raw", "ECV_2024.xlsx"))

#Cargar diccionario de variables
df_diccionario1 = load_excel(ruta_diccionario, sheetname="Etiquetas Vbles", skiprows=2)
df_diccionario2 = load_excel(ruta_diccionario, sheetname="Valores Vbles", skiprows=2)

# Análisis preliminar de la tabla principal 

'''
print("Primeras filas de df_datos:")
display(df_datos.head(10))

print("\nDimensiones de df_datos:")
print(df_datos.shape)

print("\nInformación de df_datos:")
df_datos.info()

print("\nEstadísticas descriptivas de df_datos:")
display(df_datos.describe())

print("\nPrimeras filas de columnas tipo objeto:")
display(df_datos.select_dtypes(include="object").head())
print("diccionario2:")


print("Antes de rellenar:")
display(df_diccionario2.head(10))

# Aplicar la función
df_diccionario2 = rellenar_valores(df_diccionario2, "Valor")

# Después de rellenar
print("Después de rellenar:")
display(df_diccionario2.head(10))

df_diccionario2['Valor'].apply(type)

'''
#--- PREPARACIÓN Y TRANSFORMACIÓN DE DATOS ---

# 1. Rellenar valores faltantes en la columna "Valor"
    #Renombrar columnas para estandarizar
df_diccionario2 = estandarizar_diccionario(df_diccionario2)
    # Rellenar valores
df_diccionario2 = rellenar_valores(df_diccionario2, "Valor")

    # Validar que no queden valores faltantes en "Valor"

assert df_diccionario2['Valor'].isna().sum() == 0, "Quedan valores vacios en 'Valor'"

    # Tipo de datos 
df_diccionario2.dtypes

    # Visualizacion de tabla diccionario2
print(df_diccionario2.head(10))

    ## Conclusiones
'''
- La columna `"Valor"` fue rellenada correctamente hacia adelante para completar los valores faltantes.
- No quedan valores vacíos en la columna `"Valor"`.
- La tabla `df_diccionario2` está lista para procesos posteriores, como traducción de códigos en `df_datos`.
'''

#2. Traducción de códigos en df_datos

df_datos= traducir_columnas( df_datos, df_diccionario2)
df_datos= traducir_encabezados(df_datos,df_diccionario1)

    # Visualización de df_datos ( tabla principal)
display(df_datos.head(10))

        # Crear set con todos los código originales
codigos_originales= set (df_diccionario2['Valor'].astype(str).str.strip())

        # Buscar si alguno de esos códigos sigue en df_datos
codigos_restantes = [
                    i for i in df_datos.columns
                    if df_datos[i].apply(lambda x: str(x).strip()).isin(codigos_originales).any()]

if len(codigos_restantes) == 0:
    print ("Códigos traducidos exitosamente")

else: 
    print("Quedan códigos sin traducir:", codigos_restantes)

    # Descargar BD principal traducida:

os.makedirs('data/processed', exist_ok=True)


    # Guardar base traducida
    # Ir a la carpeta raíz del proyecto (una arriba del notebook)
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))

    # Crear la ruta completa hasta data/processed
ruta_salida = os.path.join(base_dir, 'data', 'processed', 'bd_principal_traducida.xlsx')

    # Crear carpeta si no existe
os.makedirs(os.path.dirname(ruta_salida), exist_ok=True)

    # Guardar el archivo traducido
df_datos.to_excel(ruta_salida, index=False)

print(f"Archivo guardado correctamente en:\n{ruta_salida}")

 



BASE_DIR: c:\Users\User\OneDrive - CELSIA S.A E.S.P\Documentos\Aprendiendo analisis de datos\Analisis-datos-medellin


c:\Users\User\OneDrive - CELSIA S.A E.S.P\Documentos\Aprendiendo analisis de datos\Analisis-datos-medellin\src\transform.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df[columna]= df[columna].fillna(method=metodo)
c:\Users\User\OneDrive - CELSIA S.A E.S.P\Documentos\Aprendiendo analisis de datos\Analisis-datos-medellin\src\translate_codes.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_datos= df_datos.applymap(lambda x: x.strip() if isinstance(x, str) else x)


  Unnamed: 0 Valor Codigo                      Etiqueta
0        nan   A.3      1                            Sí
1        nan   A.3      2                            No
2        nan   A.4    -88                     No aplica
3        nan   A.4      1         Dirección inexistente
4        nan   A.4      2           Vivienda desocupada
5        nan   A.4      3             Personas ausentes
6        nan   A.4      4  No hay informante calificado
7        nan   A.4      5                       Rechazo
8        nan   A.4      6                No es vivienda
9        nan   A.4      7   La dirección no corresponde
Las siguientes columnas no tienen traducción: {'M.5', 'T.1', 'C.4', 'H.28', 'J.32', 'F.6', 'L.6', 'K.1', 'V.4', 'H.25', 'C.1', 'L.3', 'W.12', 'H.39', 'R.4', 'B.5', 'B.2', 'C.9', 'G.2', 'J.28', 'G.6', 'J.35', 'U.5', 'N.8', 'O.1', 'N.4', 'I.7', 'J.50', 'O.8', 'J.9', 'B.1', 'J.43', 'I.17', 'J.16', 'H.2B', 'H.40', 'F.4', 'E.9', 'L.17', 'H.11', 'J.48', 'W.10', 'R.1', 'B.3', 'K.10', 'M.3

,Formulario,Hogar,Persona,NÚMERO DE FORMULARIO,REEMPLAZO,CAUSA DEL REEMPLAZO,FECHA DE VISITA,ZONA,COMUNA O CORREGIMIENTO,Estrato de la Vivienda,...,"En este hogar, ¿Todas las situaciones anteriores se reconocen como violencia sexual infantil?","En este hogar, cuando se presenta un conflicto, generalmente la reacción es: ¿Levantan la voz y se gritan?","En este hogar, cuando se presenta un conflicto, generalmente la reacción es: ¿Utilizan golpes?","En este hogar, cuando se presenta un conflicto, generalmente la reacción es: ¿Dejan de hablarse, usan la ley del hielo?","En este hogar, cuando se presenta un conflicto, generalmente la reacción es: ¿Se calman antes de hablar para no herirse?","En este hogar, cuando se presenta un conflicto, generalmente la reacción es: ¿Dialogan para encontrar una solución?","En este hogar, cuando se presenta un conflicto, generalmente la reacción es: ¿Se enfocan en mejorar cada uno su comportamiento?",Factor de Expansión Viviendas,Factor de Expansión Hogares,Factor de Expansión Personas
0,1,1,1,1,Sí,Rechazo,04/09/2024 16:08:58,Cabecera -Urbana,Popular,Estrato 2,...,Sí,Sí,No,No,Sí,Sí,Sí,144.637096774194,145.596256684492,53.9117647058824
1,2,1,1,2,No,No aplica,04/09/2024 10:15:45,Cabecera -Urbana,Popular,Estrato 2,...,Sí,Sí,No,Sí,Sí,Sí,No,144.637096774194,145.596256684492,157.222222222222
2,3,1,1,3,No,No aplica,04/09/2024 11:03:36,Cabecera -Urbana,Popular,Estrato 2,...,Sí,No,No,No,Sí,Sí,Sí,144.637096774194,145.596256684492,53.9117647058824
3,3,1,2,3,No,No aplica,04/09/2024 11:03:36,Cabecera -Urbana,Popular,Estrato 2,...,Sí,No,No,No,Sí,Sí,Sí,144.637096774194,145.596256684492,58.7142857142857
4,3,1,3,3,No,No aplica,04/09/2024 11:03:36,Cabecera -Urbana,Popular,Estrato 2,...,Sí,No,No,No,Sí,Sí,Sí,144.637096774194,145.596256684492,110.75
5,3,1,4,3,No,No aplica,04/09/2024 11:03:36,Cabecera -Urbana,Popular,Estrato 2,...,Sí,No,No,No,Sí,Sí,Sí,144.637096774194,145.596256684492,187.833333333333
6,4,1,1,4,No,No aplica,04/09/2024 12:28:34,Cabecera -Urbana,Popular,Estrato 2,...,Sí,No,No,No,Sí,Sí,Sí,144.637096774194,145.596256684492,110.75
7,4,1,2,4,No,No aplica,04/09/2024 12:28:34,Cabecera -Urbana,Popular,Estrato 2,...,Sí,No,No,No,Sí,Sí,Sí,144.637096774194,145.596256684492,125.75
8,5,1,1,5,Sí,Personas ausentes,04/09/2024 17:32:18,Cabecera -Urbana,Popular,Estrato 2,...,Sí,Sí,No,Sí,Sí,Sí,Sí,144.637096774194,145.596256684492,86.741935483871
9,5,1,2,5,Sí,Personas ausentes,04/09/2024 17:32:18,Cabecera -Urbana,Popular,Estrato 2,...,Sí,Sí,No,Sí,Sí,Sí,Sí,144.637096774194,145.596256684492,139.296296296296


Códigos traducidos exitosamente
Archivo guardado correctamente en:
c:\Users\User\OneDrive - CELSIA S.A E.S.P\Documentos\Aprendiendo analisis de datos\Analisis-datos-medellin\data\processed\bd_principal_traducida.xlsx
